<img src="./header.png" style="width: 100%; height: 1em;" alt="banner"/>

<b>UML Extractor<b/> - 
<span>v1.1</span>
<br>
<br>
<hr>

In [ ]:
! pip install nltk
! pip install stanza
! pip install nltk

In [1]:
import stanza
#import nltk
import pandas as pd
import re

In [2]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
fr_stopwords = set(stopwords.words('french'))

[nltk_data] Error loading stopwords: <urlopen error [Errno -2] Name or
[nltk_data]     service not known>


In [3]:
#we'll dowload french model
stanza.download('fr')

ConnectionError: HTTPSConnectionPool(host='raw.githubusercontent.com', port=443): Max retries exceeded with url: /stanfordnlp/stanza-resources/main/resources_1.3.0.json (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f08666c2490>: Failed to establish a new connection: [Errno -2] Name or service not known'))

In [3]:
#Constructing Pipeline to get all text processsing tools
fr_nlp = stanza.Pipeline('fr')

2022-03-26 20:56:12 INFO: Loading these models for language: fr (French):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |
| pos       | gsd     |
| lemma     | gsd     |
| depparse  | gsd     |
| ner       | wikiner |

2022-03-26 20:56:12 INFO: Use device: cpu
2022-03-26 20:56:12 INFO: Loading: tokenize
2022-03-26 20:56:12 INFO: Loading: mwt
2022-03-26 20:56:12 INFO: Loading: pos
2022-03-26 20:56:14 INFO: Loading: lemma
2022-03-26 20:56:14 INFO: Loading: depparse
2022-03-26 20:56:17 INFO: Loading: ner
2022-03-26 20:56:26 INFO: Done loading processors!


<img src="./header.png" style="width: 100%; height: 1em;" alt="banner"/>

<b style="margin-bottom: 1em;">Functions<b/>

In [5]:
def text_to_nlp(text):
    return fr_nlp(text)

In [6]:
def statistician(text):
    tokenizer = nltk.RegexpTokenizer(r'\w+')

    stats = {'token':0, 'stats':{}, 'freq':{}}

    # Création d'un corpus de tokens par artiste
    stats['token'] = [word for word in tokenizer.tokenize(textual_text.lower()) if word not in fr_stopwords]
    stats['freq']  = fq = nltk.FreqDist(stats['token'])
    stats['stats'] = {'total': len(stats['token']), 'unique': len(fq.keys())}
    
    return stats

In [7]:
def lemmatizer(doc):
    output = []
    for i, sentence in enumerate(doc.sentences):
        w_lemma = [word.lemma for word in sentence.words]
        output.append(w_lemma)
    return output

In [8]:
def init_list_of_objects(size):
    list_of_objects = list()
    for i in range(0,size):
        list_of_objects.append(list())
    return list_of_objects

In [9]:
def preview(doc):
    for i, sent in enumerate(doc.sentences):
        print("[Sentence {}]".format(i+1))
        for word in sent.words:
            print("{:12s}\t{:12s}\t{:6s}\t{:d}\t{:12s}".format(\
              word.text, word.lemma, word.pos, word.head, word.deprel))

In [10]:
def text_parser(doc):
    tab_result = []
    for i, sent in enumerate(doc.sentences):
        sentence_size = len([i for i in sent.words])
        data = init_list_of_objects(sentence_size)
        
        for j, word in enumerate(sent.words):
            data[j].append(word.text)
            data[j].append(word.lemma)
            data[j].append(word.pos)
            data[j].append(word.deprel)
            data[j].append(word.head)
        tab_result.append({"Sentence {}".format(i+1):data})
        
    print("-"*55)
    preview(doc)
    print("-"*55)
    return tab_result

In [11]:
def cardinality_finder(sentence):
    cardinality_reper ={'un et un seul':'1,1', 'un':'1,*', 'des':'0,*', 'plusieurs':'0,*', 'au moins':'1,*'}

    for search_val in cardinality_reper.keys():
        #print(search_val)
        combined = "(" + ")|(".join(search_val) + ")"
        
        if search_val in sentence:
        #if re.match(combined, sentence):
            #print("{} -- {}".format(sentence, search_val))
            return cardinality_reper[search_val]

In [12]:
def class_drawer(sentence):
    #we make our sentence
    sentence_text = ' '.join([word[0].lower() for word in sentence])
    #let's try to identify the classes and relation
    valuable = [word for word in sentence if (word[2] == 'NOUN' or word[2] == 'VERB' or word[2] == 'DET')]
    #We start from the assumption that the classes are noun
    potential_class = [word for word in sentence if word[2] == 'NOUN']
    #Relation will be base verb of sentences
    potential_relation = [word for word in sentence if word[2] == 'VERB']    
    #Let get cardinality right now
    potential_cardinality = []
    
    for _pos, _class in enumerate(potential_class):
        seg = ''
        if _pos == 0:
            for i in range(0, sentence_text.index(_class[0])):
                seg += sentence_text[i]
            potential_cardinality.append(cardinality_finder(seg))
        else:
            for i in range(sentence_text.index(potential_class[_pos-1][0]), sentence_text.index(_class[0])):
                seg += sentence_text[i]
            potential_cardinality.append(cardinality_finder(seg))
        
            
    potential_cardinality_data = [word for word in sentence if word[2] == 'DET']
    
    verb_xcomp = [verb[1] for verb in potential_relation if verb[3] == 'xcomp']
    #print(verb_xcomp)
    if verb_xcomp == []:
        verb_xcomp = [verb[1] for verb in potential_relation if verb[3] == 'root']
    #print(verb_xcomp)
    
    return [potential_class[0][1], [potential_cardinality[0]], verb_xcomp[0] if(len(verb_xcomp) >= 0) else [], [potential_cardinality[1]], potential_class[1][1]]

In [197]:
#Step1 : text parser

In [187]:
def subject_finder(plain_text):
    doc = fr_nlp(plain_text)
    subject = []

    for word in doc.sentences[0].words:
        if 'NOUN' == word.pos:
            subject.append(word.lemma)
            
    return subject

In [269]:
def atomic_sentence_maker(plain_text, verbose=True):
    doc =  fr_nlp(plain_text)
    result = []
    
    for sentence in doc.sentences:
        #TEXT TO SVO FORMAT
        subject = []
        new_sentence = ""
        potential_verb = []
        for word in sentence.words:
            if 'VERB' in word.pos or 'AUX' in word.pos:
                potential_verb.append(word.lemma)
            if 'VERB' in word.pos or 'root' in word.deprel:
                potential_verb.insert(0, word.lemma)
            if 'VERB' in word.pos and 'xcomp' in word.deprel:
                potential_verb.insert(0, word.lemma)
            new_sentence += word.lemma+" "
            
        if len(potential_verb) == 0:
            continue
            
        divided_sentence = re.split(potential_verb[0], str(new_sentence))
        subject = subject_finder(divided_sentence[0])
        
        broken_sentence = re.split(',|;| et | ou ', str(divided_sentence[1]))
        
        for i, bloc in enumerate(broken_sentence):
            other_noun = []
            for word in fr_nlp(bloc).sentences[0].words:
                if 'NOUN' in word.pos:
                    other_noun.append(word.lemma)
            broken_sentence[i] = other_noun[0] + '.'
        
        svo = []
        for sub in subject:
            index = sub + " " + potential_verb[0] + " "
            for bloc in broken_sentence:
                svo.append(index + bloc)
                
        #SENTENCE GROUPER
        for atom in svo:
            result.append(atom)
                
        if verbose:
            #DEBUG VIEWER
            print(svo)
            print('-'*20)
            #------------
    return result

In [204]:
#helper

In [ ]:
def get_input_text():
    plain_text = str(input('Input : '))
    return doc

In [205]:
#€xtractor

In [247]:
def class_extractor(sentence_list, verbose=True):
    class_with_desc = {}
    potential_class = []
    potential_attribute = []
    
    for sentence in sentence_list:
        sentence_tab = sentence[:-1].split(' ')
        potential_class.append(sentence_tab[0])
        potential_attribute.append(sentence_tab[2])
        class_with_desc[sentence_tab[0]] = []
        
    for _class in potential_class:
        if _class in potential_attribute:
            potential_attribute.remove(_class)
        
    for sentence in sentence_list:
        sentence_tab = sentence[:-1].split(' ')
        if sentence_tab[2] in potential_attribute:
            class_with_desc[sentence_tab[0]].append(sentence_tab[2])
          
    if verbose:
        #DEBUG VIEWER
        for _key, _class in class_with_desc.items():
            print("[CLASS]\t{} : {}".format(_key, _class))
        #------------
            
    return class_with_desc

In [264]:
def relation_extractor(sentence_list, class_desc, verbose=True):
    potential_relation = []
    
    for sentence in sentence_list:
        sentence_tab = sentence[:-1].split(' ')
        if sentence_tab[0] in class_desc.keys() and sentence_tab[2] in class_desc.keys():
            potential_relation.append([sentence_tab[0], [sentence_tab[1], 'ASSOC'], sentence_tab[2]])
        
    if verbose:
        #DEBUG VIEWER
        for relation in potential_relation:
            print(relation)
        #------------
        
    return potential_relation

<img src="./header.png" style="width: 100%; height: 1em;" alt="banner"/>

<h6 style="color: red; margin-bottom: 1em;">main.py</h6><hr>

<b>GET INPUT</b>

#TEST TEXT<br>
Un répertoire contient des fichiers. Une pièce contient des murs. Les modems et claviers sont des périphériques d’entrée-sortie. Une transaction boursière est un achat ou une vente. Un compte bancaire peut appartenir à une personne physique ou morale. Un étudiants peut suivre plusieurs cours.Un répertoire contient des fichiers. Une pièce contient des murs. Un étudiants peut suivre plusieurs cours.

Les modems et claviers sont des périphériques d’entrée-sortie. Une transaction boursière est un achat ou une vente. Un répertoire contient des fichiers. Une pièce contient des murs. Un étudiant a un nom, un prénom et un matricule. Les professeurs ont un code et une matière a enseigné.

Un cour est enseigné par un professeur.


In [275]:
plain_text = get_input_text()
svo = atomic_sentence_maker(plain_text)

Input : Un cour est enseigné par un professeur. UN cour possède un id et un intutilé. le professeur a un email et un pot_de_passe.
['cour enseigner professeur.']
--------------------
['cour posséder id.', 'cour posséder intutilé.']
--------------------
['professeur avoir email.', 'professeur avoir pot_de_passe.']
--------------------


In [276]:
class_desc = class_extractor(svo)

[CLASS]	cour : ['id', 'intutilé']
[CLASS]	professeur : ['email', 'pot_de_passe']


In [277]:
relation_desc = relation_extractor(svo, class_desc)

['cour', ['enseigner', 'ASSOC'], 'professeur']


<b>LEMMATIZATION</b>

In [266]:
doc = fr_nlp("Un cour est enseigné par un professeur.")

In [267]:
lemmatizer(doc)

[['un', 'cour', 'être', 'enseigner', 'par', 'un', 'professeur', '.']]

<b>DOCS PARSER</b>

In [268]:
concept_list = text_parser(doc)

-------------------------------------------------------
[Sentence 1]
Un          	un          	DET   	2	det         
cour        	cour        	NOUN  	4	nsubj:pass  
est         	être        	AUX   	4	aux:pass    
enseigné    	enseigner   	VERB  	0	root        
par         	par         	ADP   	7	case        
un          	un          	DET   	7	det         
professeur  	professeur  	NOUN  	4	obl:agent   
.           	.           	PUNCT 	4	punct       
-------------------------------------------------------


In [98]:
for i, sentence in enumerate(concept_list):
    print(class_drawer(sentence['Sentence {}'.format(i+1)]))

['étudiant', ['1,*'], 'avoir', ['1,*'], 'nom']


In [99]:
#GENERAL VAR
ATTRIBUTE = []
with open('./src/attribute_fr_dataset.txt') as attrib_file:
    for attribute in attrib_file:
        ATTRIBUTE.append(attribute.strip())
        
DESIGN_ELEMENT = []
with open('./src/system_design_keyword.txt') as attrib_file:
    for attribute in attrib_file:
        DESIGN_ELEMENT.append(attribute.strip())

AGREGATION_LIST = []
with open('./src/system_design_keyword.txt') as attrib_file:
    for attribute in attrib_file:
        AGREGATION_LIST.append(attribute.strip())


In [20]:
ATTRIBUTE

['id', 'nom', 'prénom', 'prix', 'matricule', 'adresse', '']

In [21]:
def class_extractor(sentence):
    sentence_text = ' '.join([word[0].lower() for word in sentence])
    new_doc = text_to_nlp(sentence_text)
    tokens = []

    for token in new_doc.sentences[0].tokens:
        tokens.append(token)
        
    #print(tokens)
        
    
    sentence = [word for word in sentence if  word[2] not in "PUNCT"]
    sentence = [word for word in sentence if  word[1].lower() not in fr_stopwords]    
    sentence = [word for word in sentence if  word[1].lower() not in DESIGN_ELEMENT]
    sentence = [word for word in sentence if  word[1].lower() not in ATTRIBUTE]
    sentence = [word for i, word in enumerate(sentence) if "PER" not in tokens[i].ner or "MISC" not in tokens[i].ner]
    sentence = [word for word in sentence if  word[2] not in "DET" and word[2] not in "VERB"]
    return sentence

In [22]:
for i, sentence in enumerate(concept_list):
    print(class_extractor(sentence['Sentence {}'.format(i+1)]))

[['étudiants', 'étudiant', 'NOUN', 'nsubj', 3], ['cours', 'cours', 'NOUN', 'obj', 4]]


In [22]:
def relation_extractor(sentence):
    potential_relation = [word for word in sentence if word[2] == 'VERB']    
    
    
    verb_xcomp = [verb[1] for verb in potential_relation if verb[3] == 'xcomp']
    #print(verb_xcomp)
    if verb_xcomp == []:
        verb_xcomp = [verb[1] for verb in potential_relation if verb[3] == 'root']
    #print(verb_xcomp)
    
    nature = ['ASSOCIATION', 'AGGREGATION', 'COMPOSITION']
    
    
    return [verb_xcomp[0], nature] if(len(verb_xcomp) >= 0) else []

In [23]:
for i, sentence in enumerate(concept_list):
    print(relation_extractor(sentence['Sentence {}'.format(i+1)]))

NameError: name 'nature' is not defined

In [25]:
#CLASS EXTRACTION RULES
# ❌️ if word accur on time and it frequency lower than 2%, --> ignore. -- delete rules
# ✅️ if word related to design elements[application, system, data], --> ignore.
# ✅️ if word related to poeple name or location, --> ignore.
# if word is an attribute[id, name, phone_number], --> ignore.
# ✅️ if word is a verb or det then, --> ignore.
# ------------------------------------------------------
#                 user        <-- hypernym
#                  |               
#              --------- 
#               |     |
#         Student    Teacher  <-- hyponym
#            |           |
#     ----------------  --------------------------
#      |     |       |                   |      |   
#  primary  college  university
# ------------------------------------------------------
# if word is found in hight level tree of hyponym tree, --> ignore.
# else it probably a [CLASS].
# if we're in Noun phrase case (Noun+Noun), if 2nd Word == attrib then 1rst is class.
# 
# 
# 
# 
# 

#ATTRIBUTE EXTRACTION RULES
# In case of noun phrase (Noun+Noun) if we've underscore mark between two noun then
# the 1rst noun is an attribute and thee 2nd is the class.
#
# If word can have only one value then it's an attribute
# based on predefined popular attributes list[ID, address, name, email, ...]
# 
# 
# 
# 
# 

#RELATIONSHIP EXTRACTION RULES
# -- OUR RELATION HAVE TO BE A VERB --
# 
# (1) if we've the diagram [class - word - class] and word is verb then, --> AASOCIATION
# if (1) and verb is equal to [aggregation word] then, --> AGGREGATION OR COMPISITION
# same for DEPENDENCIES 
# 
# if CLASS1 - VERB - CLASS2 AND CLASS3 then,    --> { CLASS1 IN RELATION WHITH CLASS2
#                                                   { CLASS1 IN RELATION WHITH CLASS3
#
# if CLASS1 - VERB - CLASS2 AND NOT CLASS3 then,--> { CLASS1 IN RELATION WHITH CLASS2 ONLY
# 
# 
# 
# 
# 

#CARDINALITY RULES
# Use pretrained model to identify cardinality
# OR
# define set of rules
# 
# 
# 
# 
# 